<a href="https://colab.research.google.com/github/daliaydom/Tarea2_AprendizajeProfundo/blob/main/T2Ejercicio3CNNvsRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 3: Reconocimiento de acciones humanas usando RNNs y CNNs

Dalia Yvette Domínguez Jiménez
---
---

Se entrena un modelo basado en RNNs, como se vió en clase. También se implementa y entrena una CNN con una capa Conv1D, para reconocimiento de acciones humanas (HAR) en el conjunto [UCF11](https://www.crcv.ucf.edu/data/UCF_YouTube_Action.php). 

<img src="https://www.crcv.ucf.edu/data/youtube_snaps.jpg" width=800/>

Se utilizan las características convolucionales vistas en clase.

## 1 Preparación

### 1.1 Bibliotecas

In [1]:
# Colab
# https://github.com/TylerYep/torchinfo
!pip install torchinfo
# https://zarr.readthedocs.io/en/stable/
!pip install zarr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 185 kB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 6.7 MB/s 
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5050 sha256=2f45418379c729709b4efcb622754e04485e4b67f8ee58ce8630311f809199f8
  Stored in directory: /root/.cache/pip/wheels/12/1c/38/0def51e15add93bff3f4bf9c248b94db0839b980b8535e72a0
Successfully built asciitree


In [2]:
# sistema de archivos
import os
# funciones aleatorias
import random
# descomprimir
import tarfile
# sistema de archivos
from os.path import join

# arreglos multidimensionales
import numpy as np
# redes neuronales
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets.utils as tvu
# almacenamiento de arreglos multidimensionales
import zarr
#redes
from torch.utils.data import DataLoader, random_split
# inspección de arquitectura
from torchinfo import summary

# directorio de datos
DATA_DIR = '../data'

# tamaño del lote
BATCH_SIZE = 32
# tamaño del vector de características
FEAT_SIZE = 1024

# reproducibilidad
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch_gen = torch.manual_seed(SEED)

## 2 Datos

### 2.1 Conjunto de datos

In [3]:
class UCF11:

    def __init__(self, root, download=False):
        self.root = root
        self.zarr_dir = join(root, 'ucf11.zarr')
        if download:
            self.download()
        self.z = zarr.open(self.zarr_dir, 'r')
        self.paths = list(self.z.array_keys())
        
    def __getitem__(self, i):
        arr = self.z[self.paths[i]]
        x = np.array(arr)
        y = np.array(arr.attrs['y'], dtype=np.int64)
        return x, y

    def __len__(self):
        return len(self.paths)
    
    def _check_integrity(self):
        return os.path.isdir(self.zarr_dir)
    
    def _extract(self, root, filename):
        tar = tarfile.open(join(root, filename), "r:gz")
        tar.extractall(root)
        tar.close()

    def download(self):
        if self._check_integrity():
            print('Files already downloaded and verified')
            return
        tvu.download_url(
            url='https://cloud.xibalba.com.mx/s/apYrNA4iM4K65o7/download',
            root=self.root,
            filename='ucf11.zarr.tar.gz',
            md5='c8a82454f9ec092d00bcd99c849e03fd'
        )
        self._extract(self.root, 'ucf11.zarr.tar.gz')

### 2.2 Instancia del conjunto y partición

In [4]:
ds = UCF11(join(DATA_DIR, 'ucf11'), True)
x, y = ds[0]
print(f'x shape={x.shape} dtype={x.dtype}')
print(f'x [0][:5]={x[0][:5]}')
print(f'y shape={y.shape} dtype={y.dtype} {y}')
print(f'y {y}')

  0%|          | 0/53436566 [00:00<?, ?it/s]

x shape=(10, 1024) dtype=float32
x [0][:5]=[0.00022111 0.00368518 0.00314753 0.00201778 0.09296297]
y shape=() dtype=int64 0
y 0


In [5]:
trn_size = int(0.8 * len(ds))
tst_size = len(ds) - trn_size
trn_ds, tst_ds = random_split(ds, [trn_size, tst_size])
len(trn_ds), len(tst_ds)

(1279, 320)

### 2.3 Cargadores de datos

In [6]:
trn_dl = DataLoader(
    # conjunto
    trn_ds,
    # tamaño del lote
    batch_size=BATCH_SIZE,
    # desordenar
    shuffle=True,
    # procesos paralelos
    num_workers=2
)
tst_dl = DataLoader(
    # conjunto
    tst_ds,
    # tamaño del lote
    batch_size=BATCH_SIZE,
    # desordenar
    shuffle=True,
    # procesos paralelos
    num_workers=2
)

In [7]:
x, y = next(iter(trn_dl))
print(f'x shape={x.shape} dtype={x.dtype}')
print(f'y shape={y.shape} dtype={y.dtype}')

x shape=torch.Size([32, 10, 1024]) dtype=torch.float32
y shape=torch.Size([32]) dtype=torch.int64


## 3 Modelo

<!-- Torchvision provee una familia de [modelos](https://pytorch.org/docs/1.6.0/torchvision/models.html#classification) preentrenados en ImageNet. Usaremos [Shufflenet V2](https://arxiv.org/abs/1807.11164), una arquitectura eficiente para clasificación de imágenes.  -->

### 3.1 Definiciones de arquitecturas

#### 3.1.1 CNN

Definiremos el modelo como una cama Conv1D. Es habitual definir las capas de la CNN en grupos de dos para que el modelo tenga una buena oportunidad de aprender características de los datos de entrada. Las CNN aprenden muy deprisa, por lo que a veces se agrega una capa de Dropout que tiene por objeto ayudar a ralentizar el proceso de aprendizaje y, con suerte, dar lugar a un mejor modelo final. La capa de pooling reduce las características aprendidas a 1/4 de su tamaño, consolidándolas sólo en los elementos más esenciales.

Después de la CNN y el pooling, las características aprendidas se aplanan a un vector largo y pasan por una capa totalmente conectada antes de la capa de salida utilizada para hacer una predicción. La capa totalmente conectada proporciona idealmente un amortiguador entre las características aprendidas y la salida con la intención de interpretar las características aprendidas antes de hacer una predicción.

La definición del modelo es la siguiente.

In [8]:
class CNN(nn.Module):

    def __init__(self, in_channels=10, out_channels=64, num_classes=11):
        super(CNN, self).__init__()
        self.num_feats = 32*512
        # O1,O2=out_channels=out_channels
        self.cnn= nn.Sequential(
            # bloque conv1
            # [N, 1, 10, 1024] => [N, 1, O 1024]
            nn.Conv1d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,padding=1),
            # nn.Conv1d(in_channels=O1,out_channels=O2,kernel_size=3,bias=True),
            # nn.Dropout(0.5),
            # [N, 1, O, 1024] => [N, 1, O/2, 512]
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.flatten = nn.Flatten()

        self.cls = nn.Linear(self.num_feats, num_classes)

    # metodo para inferencia
    def forward(self, x):
        x = self.cnn(x)
        x = self.flatten(x)
        x = self.cls(x)
        return x

In [9]:
modelCNN = CNN()
print(modelCNN)

CNN(
  (cnn): Sequential(
    (0): Conv1d(10, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (cls): Linear(in_features=16384, out_features=11, bias=True)
)


In [10]:
summary(modelCNN, (1, 10, 1024), device='cpu', verbose=0)

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [1, 11]                   --
├─Sequential: 1-1                        [1, 32, 512]              --
│    └─Conv1d: 2-1                       [1, 64, 1024]             1,984
│    └─MaxPool2d: 2-2                    [1, 32, 512]              --
├─Flatten: 1-2                           [1, 16384]                --
├─Linear: 1-3                            [1, 11]                   180,235
Total params: 182,219
Trainable params: 182,219
Non-trainable params: 0
Total mult-adds (M): 2.21
Input size (MB): 0.04
Forward/backward pass size (MB): 0.52
Params size (MB): 0.73
Estimated Total Size (MB): 1.29

#### 3.1.2 RNN

Se utilizó la arquitectura que se revisó en clase

In [11]:
class RNN(nn.Module):

    def __init__(self, input_size=1024, hidden_size=55, num_classes=11):
        super().__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size,
                          num_layers=1, batch_first=True)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Batch, Seq, Feats, Hidden
        # [B, S, F] => [B, F, S]
        x = x.movedim(1, 2)
        # [B, F, S]
        x = self.bn(x)
        # [B, F, S] => [B, S, F]
        x = x.movedim(1, 2)
        # [B, S, F] => [B, S, H]
        x, _ = self.rnn(x)
        # [B, S, H] => [B, H]
        # toma el último paso, participación 1
        x = x[:, -1, :]
        # [B, H] = [B, 11]
        x = self.cls(x)
        return x

In [12]:
model = RNN().eval()
model(torch.zeros(1, 10, 1024)).shape

torch.Size([1, 11])

### 3.2 Inspección de arquitectura

In [13]:
summary(model, (1, 10, 1024), device='cpu', verbose=0)

Layer (type:depth-idx)                   Output Shape              Param #
RNN                                      [1, 11]                   --
├─BatchNorm1d: 1-1                       [1, 1024, 10]             2,048
├─GRU: 1-2                               [1, 10, 55]               178,365
├─Linear: 1-3                            [1, 11]                   616
Total params: 181,029
Trainable params: 181,029
Non-trainable params: 0
Total mult-adds (M): 1.79
Input size (MB): 0.04
Forward/backward pass size (MB): 0.09
Params size (MB): 0.72
Estimated Total Size (MB): 0.85

## 4 Entrenamiento

### 4.1 Ciclo de entrenamiento

In [14]:
# optimizador
def train(model,trn_dl,tst_dl,lr=1e-3,EPOCHS = 10):
  opt = optim.Adam(model.parameters(), lr=lr)

  # ciclo de entrenamiento
  for epoch in range(EPOCHS):

      # modelo en modo de entrenamiento
      model.train()
      
      # entrenamiento de una época
      for x, y_true in trn_dl:
          # hacemos inferencia para obtener los logits
          y_lgts = model(x)
          # calculamos la pérdida
          loss = F.cross_entropy(y_lgts, y_true)
          # vaciamos los gradientes
          opt.zero_grad()
          # retropropagamos
          loss.backward()
          # actulizamos parámetros
          opt.step()

      # desactivamos temporalmente la gráfica de cómputo
      with torch.no_grad():

          # modelo en modo de evaluación
          model.eval()
          
          losses, accs = [], []
          # validación de la época
          for x, y_true in tst_dl:
              # hacemos inferencia para obtener los logits
              y_lgts = model(x)
              # calculamos las probabilidades
              y_prob = F.softmax(y_lgts, 1)
              # obtenemos la clase predicha
              y_pred = torch.argmax(y_prob, 1)
              
              # calculamos la pérdida
              loss = F.cross_entropy(y_lgts, y_true)
              # calculamos la exactitud
              acc = (y_true == y_pred).type(torch.float32).mean()

              # guardamos históricos
              losses.append(loss.item() * 100)
              accs.append(acc.item() * 100)

          # imprimimos métricas
          loss = np.mean(losses)
          acc = np.mean(accs)
          print(f'E{epoch:2} loss={loss:6.2f} acc={acc:.2f}')

### 4.2 Entrenamiento de CNN

Resultados de la perdida y la precisión en el conjunto de entrenamiento

In [15]:
train(modelCNN,trn_dl,tst_dl,lr=1e-3,EPOCHS = 15)

E 0 loss=229.66 acc=13.12
E 1 loss=206.54 acc=31.88
E 2 loss=182.46 acc=37.81
E 3 loss=179.76 acc=39.38
E 4 loss=179.95 acc=40.31
E 5 loss=166.30 acc=43.12
E 6 loss=164.57 acc=43.75
E 7 loss=161.39 acc=46.25
E 8 loss=173.69 acc=44.06
E 9 loss=165.77 acc=45.94
E10 loss=162.27 acc=46.56
E11 loss=161.53 acc=50.00
E12 loss=159.30 acc=49.06
E13 loss=162.94 acc=48.44
E14 loss=164.50 acc=48.75


### 4.3 Entrenamiento de RNN

Resultados de la perdida y la precisión en el conjunto de entrenamiento

In [16]:
train(model,trn_dl,tst_dl,lr=1e-3,EPOCHS = 15)

E 0 loss=217.55 acc=28.75
E 1 loss=200.82 acc=36.25
E 2 loss=188.95 acc=40.62
E 3 loss=181.55 acc=42.50
E 4 loss=174.82 acc=42.50
E 5 loss=169.09 acc=46.88
E 6 loss=165.31 acc=46.25
E 7 loss=165.09 acc=45.94
E 8 loss=159.62 acc=48.12
E 9 loss=157.02 acc=47.19
E10 loss=159.44 acc=47.19
E11 loss=156.01 acc=50.62
E12 loss=157.08 acc=51.25
E13 loss=156.70 acc=48.75
E14 loss=156.20 acc=50.94


## 5. Discusión

Ambos modelos contienen un número de parámetros del mismo orden, lo que provoca que ambos modelos se entrenen en un tiempo muy parecido. Se utilizaron 15 epocas para entrenar los modelos y no hay una mejoría significativa en la precisión después de 11 epócas. La RNN aprennde más rápido que la CNN, aunque ambos modelos alcanzan una preción muy parecida en la época número 11. Por un lado la RNN captura y aprende direncatamente de la secuncia de imágenes de un video, mientras que la CNN lo hace indirectamente a través de la convolución, a pesar de esto último, la CNN aprende con gran velocidad, pues en la primera época tiene una precisión de 13.12 y alcanza la misma precisión que la RNN.